In [1]:
from utils import (
    LoveNumbersHyperParameters,
    load_base_model,
    parameters_path,
    Earth_radius,
    elastic_Love_numbers_computing,
)
from numpy import ones

Love_numbers_hyper_parameters: LoveNumbersHyperParameters = load_base_model(
    name="Love_numbers_hyper_parameters", path=parameters_path, base_model_type=LoveNumbersHyperParameters
)
Love_numbers_hyper_parameters.load()

In [3]:
from utils import RealDescription

# Generates real description using new elasticity description.
real_description = RealDescription(
    id="test-Asthenosphere",
    below_ICB_layers=Love_numbers_hyper_parameters.real_description_parameters.below_ICB_layers,
    below_CMB_layers=Love_numbers_hyper_parameters.real_description_parameters.below_CMB_layers,
    splines_degree=Love_numbers_hyper_parameters.real_description_parameters.splines_degree,
    radius_unit=(
        Love_numbers_hyper_parameters.real_description_parameters.radius_unit
        if Love_numbers_hyper_parameters.real_description_parameters.radius_unit
        else Earth_radius
    ),
    real_crust=Love_numbers_hyper_parameters.real_description_parameters.real_crust,
    n_splines_base=10,
    profile_precision=10,
    radius=(
        Love_numbers_hyper_parameters.real_description_parameters.radius_unit
        if Love_numbers_hyper_parameters.real_description_parameters.radius_unit
        else Earth_radius
    ),
    load_description=False,
    anelasticity_model_from_name="test-low-viscosity-Asthenosphere",
)

In [4]:
from utils import Result

# Verifies elastic results.
degrees = [1, 2]
elastic_Love_numbers = elastic_Love_numbers_computing(
    y_system_hyper_parameters=Love_numbers_hyper_parameters.y_system_hyper_parameters,
    degrees=degrees,
    real_description=real_description,
)

elastic_result = Result(hyper_parameters=Love_numbers_hyper_parameters)
elastic_result.update_values_from_array(result_array=elastic_Love_numbers, degrees=degrees)
elastic_result.values

{<Direction.radial: 0>: {<BoundaryCondition.load: 0>: array([[-0.2859902 +0.j],
         [-0.99145273+0.j]]),
  <BoundaryCondition.shear: 1>: array([[0.        +0.j],
         [0.06051891+0.j]]),
  <BoundaryCondition.potential: 2>: array([[0.        +0.j],
         [0.60329666+0.j]])},
 <Direction.tangential: 1>: {<BoundaryCondition.load: 0>: array([[0.10360985+0.j],
         [0.04669886+0.j]]),
  <BoundaryCondition.shear: 1>: array([[0.       +0.j],
         [0.6340226+0.j]]),
  <BoundaryCondition.potential: 2>: array([[0.        +0.j],
         [0.16773665+0.j]])},
 <Direction.potential: 2>: {<BoundaryCondition.load: 0>: array([[ 0.       +0.j],
         [-0.6106057+0.j]]),
  <BoundaryCondition.shear: 1>: array([[0.        +0.j],
         [0.16773667+0.j]]),
  <BoundaryCondition.potential: 2>: array([[0.        +0.j],
         [0.59598762+0.j]])}}